**1. Підготовка середовища**

In [1]:
# !pip install transformers
# !pip install datasets
# !pip install bertviz
# !pip install umap-learn
# !pip install wandb

In [2]:
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# torch.cuda.empty_cache()

**2. Вибір попередньо навченої LLM та завантаження набору даних:**

In [5]:
from transformers import BertTokenizer, BertModel
model_ckpt = 'bert-base-multilingual-uncased'
tokenizer = BertTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#This is the first of its kind toxicity classification dataset for the Ukrainian language. The datasets was obtained semi-automatically by toxic keywords filtering. 
dataset = load_dataset("ukr-detect/ukr-toxicity-dataset-seminatural")

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'tags'],
        num_rows: 12682
    })
    validation: Dataset({
        features: ['text', 'tags'],
        num_rows: 4227
    })
    test: Dataset({
        features: ['text', 'tags'],
        num_rows: 4214
    })
})

In [8]:
import math

def filter_nulls(example):
    return example["tags"] is not None and not math.isnan(example["tags"])

dataset = dataset.filter(filter_nulls)

def rename_and_convert(example):
    example["label"] = int(example["tags"])
    return example

dataset = dataset.map(rename_and_convert)
dataset = dataset.remove_columns("tags")

In [9]:
df = pd.DataFrame(dataset['train'][:])

In [10]:
df

,text,label
0,"тут посони не знають нічого про лєщєнко, #меді...",1
1,"як казав мій дід:""позакривали пиздаки""",1
2,"Не покидає відчуття, що щось я зробила не так....",0
3,— Сідай !,0
4,Чистка грибів - справа сімейнаpic.twitter.com/...,0
...,...,...
12601,"— Чула , серденько , чула ...",0
12602,Із цим поняттям ви вже стикалися .,0
12603,колись я відпиздила однокласника іграшковим по...,1
12604,"also, ті ""люди старшого віку"" ніхуя в своєму ж...",1


In [11]:
df['label_tags'] = df['label'].apply(lambda x: 'toxic' if x==1 else 'non-toxic')

In [12]:
df

,text,label,label_tags
0,"тут посони не знають нічого про лєщєнко, #меді...",1,toxic
1,"як казав мій дід:""позакривали пиздаки""",1,toxic
2,"Не покидає відчуття, що щось я зробила не так....",0,non-toxic
3,— Сідай !,0,non-toxic
4,Чистка грибів - справа сімейнаpic.twitter.com/...,0,non-toxic
...,...,...,...
12601,"— Чула , серденько , чула ...",0,non-toxic
12602,Із цим поняттям ви вже стикалися .,0,non-toxic
12603,колись я відпиздила однокласника іграшковим по...,1,toxic
12604,"also, ті ""люди старшого віку"" ніхуя в своєму ж...",1,toxic


In [13]:
classes = df['label_tags'].unique().tolist()
classes[0], classes[1] = classes[1], classes[0]
classes

['non-toxic', 'toxic']

**3.Попередня обробка даних**

In [14]:
# Функція для токенізації
def tokenize_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True)

# Токенізація датасету
tokenized_datasets = dataset.map(tokenize_function, batched=True, batch_size=None)

In [15]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12606
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4202
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4202
    })
})

**4. Налаштування (finetuning) моделі:**

In [16]:
from transformers import AutoModelForSequenceClassification

num_labels = len(classes)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt,
                                                           num_labels = num_labels).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
import wandb
wandb.init(mode = "disabled")

In [18]:
from transformers import TrainingArguments, Trainer


batch_size = 32
model_name = "bert-base-multilingual-cased-ukr-emotion" #bert-base-multilingual-uncased

training_args = TrainingArguments(output_dir = model_name,
                                  num_train_epochs = 3,
                                  learning_rate = 2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  weight_decay = 0.01,
                                  evaluation_strategy = 'epoch',
                                  disable_tqdm = False,
                                  )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average='weighted')
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

In [20]:
from transformers import Trainer

trainer = Trainer(model = model,
                  args = training_args,
                  compute_metrics = compute_metrics,
                  train_dataset = tokenized_datasets['train'],
                  eval_dataset = tokenized_datasets['validation'],
                  tokenizer = tokenizer)

<ipython-input-20-07557c0d69ef>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model = model,


**5. Навчання та оцінка ефективності модел**

In [21]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.048734,0.987863,0.987862
2,0.142600,0.037099,0.993337,0.993337
3,0.024600,0.039643,0.992147,0.992147


TrainOutput(global_step=1182, training_loss=0.07306229765645139, metrics={'train_runtime': 2361.6638, 'train_samples_per_second': 16.013, 'train_steps_per_second': 0.5, 'total_flos': 6471603878719320.0, 'train_loss': 0.07306229765645139, 'epoch': 3.0})

In [22]:
preds_outputs = trainer.predict(tokenized_datasets['test'])
preds_outputs.metrics

{'test_loss': 0.031149307265877724,
 'test_accuracy': 0.9940504521656354,
 'test_f1': 0.9940504430674755,
 'test_runtime': 85.084,
 'test_samples_per_second': 49.387,
 'test_steps_per_second': 1.551}

In [43]:
#Приклад перевірки класифікації токсичних слів
result = ['привіт! як справи загалом?', 'цей чоловік занадтно класний','це так пиздато','йобана русня','путін хуйло']

for text in result:
  input_encoded = tokenizer(text, return_tensors='pt').to(device)
  with torch.no_grad():
    outputs = model(**input_encoded)

  logits = outputs.logits
  pred = torch.argmax(logits, dim=1).item()
  print(f'{text} Predict: {pred}, {classes[pred]}')

привіт! як справи загалом? Predict: 0, non-toxic
цей чоловік занадтно класний Predict: 0, non-toxic
це так пиздато Predict: 1, toxic
йобана русня Predict: 1, toxic
путін хуйло Predict: 1, toxic


**7. Документування результатів**

1. Для покращення результатів було збільшено batch_size з 8 до 32(з підключеним GPU)
2. Також зменшено к-сть епох до 3(Модель BERT зазвичай добре узагальнює за 3 епохи.)

In [31]:
model.save_pretrained(".config/toxicity_model")
tokenizer.save_pretrained(".config/toxicity_model")

('.config/toxicity_model/tokenizer_config.json',
 '.config/toxicity_model/special_tokens_map.json',
 '.config/toxicity_model/vocab.txt',
 '.config/toxicity_model/added_tokens.json')